In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import tqdm_notebook

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
ls 

drive/  sample_data/


In [4]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix'

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [5]:
ls data 

men_shoes_prices.csv


In [0]:
df = pd.read_csv('data/men_shoes_prices.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
    X = df[ feats ].values
    y = df['prices_amountmin'].values

    scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
    return np. mean(scores), np.std(scores)

In [8]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [9]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [10]:
df.head

<bound method NDFrame.head of                          id asins  ... weight brand_cat
0      AVpfHrJ6ilAPnD_xVXOI   NaN  ...    NaN         0
1      AVpfHrJ6ilAPnD_xVXOI   NaN  ...    NaN         0
2      AVpfHsWP1cnluZ0-eVZ7   NaN  ...    NaN         1
3      AVpfHsWP1cnluZ0-eVZ7   NaN  ...    NaN         1
4      AVpfHsWP1cnluZ0-eVZ7   NaN  ...    NaN         1
...                     ...   ...  ...    ...       ...
18275  AVpfdSjlilAPnD_xcGPm   NaN  ...    NaN      1731
18276  AVpf3bFWilAPnD_xjrQ2   NaN  ...    NaN       299
18277  AVpf0fJXLJeJML43EVe9   NaN  ...    NaN       399
18278  AVpf0fJXLJeJML43EVe9   NaN  ...    NaN       399
18279  AVpf0fJXLJeJML43EVe9   NaN  ...    NaN       399

[18280 rows x 49 columns]>

In [11]:
df.features.head()

0    [{"key":"Gender","value":["Men"]},{"key":"Shoe...
1    [{"key":"Gender","value":["Men"]},{"key":"Shoe...
2    [{"key":"Gender","value":["Men"]},{"key":"Colo...
3    [{"key":"Gender","value":["Men"]},{"key":"Colo...
4    [{"key":"Gender","value":["Men"]},{"key":"Colo...
Name: features, dtype: object

In [0]:
def parse_features(x):
    output_dict = {}
    if str(x) == 'nan': return output_dict

    features = literal_eval(x.replace('\\"', '"'))
    for item in features:
        key = item['key'].lower().strip()
        value = item['value'][0].lower().strip()

        output_dict[key] = value
    return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [13]:
keys = set()


df['features_parsed'].map( lambda x: keys.update(x.keys() ))

len(keys)

476

In [14]:
def get_name_feat(key):
    return 'feat_' + key

for key in tqdm_notebook(keys):
    df[get_name_feat(key)] = df.features_parsed.map(lambda feats: key in feats[key] if key in feats else np.nan)

In [15]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_lens color family', 'feat_light transmission (vlt)',
       'feat_is weather-resistant', 'feat_uv rating', 'feat_wheeled',
       'feat_wash', 'feat_leather grade', 'feat_materials', 'feat_part number',
       'feat_number of pockets'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:
    keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [17]:
{k:v for k,v in keys_stat.items() if v > 30} 

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]    

In [0]:
feats_cat = [x for x in df.columns if '_cat' in x]

In [28]:
df.weight.unique()

array([nan, '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 Kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 Kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 Kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 Kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '71.

In [0]:

def convert_weight_to_g(w):
  if not isinstance(w, str):
    return -1

  unit_w = w.split(" ")[-1]
  number_w = float(w.split(" ")[0])

  if unit_w == "pounds" or unit_w == "lb" or unit_w == "lbs":
    return int(number_w * 454)
  elif unit_w == "g":
    return int(number_w)
  elif unit_w == "Kg":
    return int(number_w * 1000)
  elif unit_w == "ounces":
    return int(number_w * 28.3)
  else: return -1

In [31]:
df['weight_int'] = df['weight'].map(convert_weight_to_g)
df[['weight_int', 'weight']].sample(40)

,weight_int,weight
9072,-1,NaN
964,-1,NaN
4066,-1,NaN
16614,-1,NaN
8090,-1,NaN
10190,-1,NaN
4770,-1,NaN
9235,2270,5.0 lbs
17669,-1,NaN
6108,-1,NaN


In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat', 'feat_metal type_cat', 'feat_size_cat', 'feat_occasion_cat', 'weight_int']
model = RandomForestRegressor(max_depth=25, n_estimators=800, min_samples_split=4, min_samples_leaf=2)
result = run_model(feats, model)

In [35]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=24, n_estimators=800, min_samples_split=4, min_samples_leaf=2)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
#print(perm)
eli5.show_weights(perm, feature_names=feats)

(-46.520331859869586, 4.351908040397204)


Weight,Feature
0.9312 ± 0.0122,brand_cat
0.2948 ± 0.0206,feat_color_cat
0.0928 ± 0.0047,feat_gender_cat
0.0912 ± 0.0071,feat_material_cat
0.0699 ± 0.0064,feat_style_cat
0.0339 ± 0.0035,feat_size_cat
0.0265 ± 0.0035,feat_brand_cat
0.0228 ± 0.0020,weight_int
0.0189 ± 0.0021,feat_metal type_cat
0.0116 ± 0.0022,feat_occasion_cat


In [36]:
ls 

data/  HelloGithub.ipynb  LICENSE  matrix_one/  README.md
